In [1]:
# Install the required package
!pip install bert-for-tf2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30531 sha256=009bb2c87a616d72d0889269c47535de13917390698f5cfc09a34f4ecd749056
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19471 sha256=06cff0808f8261a8dce887ca8fb6dae2663108fddeccb7fc79847b702965271a
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7910 sha256=380922a119e05ff52cc60a83431dbb1e4024c729b102d50a6f5d0d387268d17a
  Sto

In [2]:

# Import modules
import os
import bert
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
pd.set_option('display.max_colwidth',1000)

TensorFlow Version: 2.12.0
Hub version:  0.13.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Presidential/IMDB Dataset.csv")

In [5]:
# Take a peek at the dataset
df.head(5)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the diffe

In [6]:
# Identify missing values
df.apply(lambda x: sum(x.isnull()), axis=0)

review       0
sentiment    0
dtype: int64

In [7]:
# Check the target class balance
df["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
MAX_SEQ_LEN = 0
seq_len = []
for sent in df["review"]:
    if len(sent) > MAX_SEQ_LEN: MAX_SEQ_LEN = len(sent)
    seq_len.append(len(sent))
print(MAX_SEQ_LEN)
print(sum(seq_len)/len(df))

13704
1309.43102


In [9]:
# Example of BERT Tokenizer Setup
# https://www.tensorflow.org/official_models/fine_tuning_bert
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file = os.path.join(gs_folder_bert, "vocab.txt"), do_lower_case=True)
print("Vocab size:", len(tokenizer.vocab))
tokens = tokenizer.tokenize("Hello TensorFlow!")
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
print(tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]']))

Vocab size: 30522
['hello', 'tensor', '##flow', '!']
[7592, 23435, 12314, 999]
[101, 102]


In [10]:
def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)
    print("Vocab size:", len(tokenizer.vocab))
    return tokenizer

In [11]:
def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)
    print("Vocab size:", len(tokenizer.vocab))
    return tokenizer

In [12]:
def get_ids(tokens, tokenizer, MAX_SEQ_LEN):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids
# checkfunality of get_ids
# print(get_ids(tokenizer.tokenize("Hello TensorFlow!"), tokenizer, 500))

In [13]:

def get_masks(tokens, MAX_SEQ_LEN):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1] * len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))

In [14]:
def get_segments(tokens, MAX_SEQ_LEN):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))

# checkfunality of get_masks
# print(get_segments(tokenizer.tokenize("Hello TensorFlow!"), 500))  
     

In [15]:
def create_single_input(sentence, tokenizer, max_len):
    """Create an input from a sentence"""
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[:max_len] # max_len = MAX_SEQ_LEN - 2, why -2 ? ans: reserved for [CLS] & [SEP]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
    return get_ids(stokens, tokenizer, max_len+2), get_masks(stokens, max_len+2), get_segments(stokens, max_len+2)

In [16]:
def convert_sentences_to_features(sentences, tokenizer, MAX_SEQ_LEN):
    """Convert sentences to features: input_ids, input_masks and input_segments"""
    input_ids, input_masks, input_segments = [], [], []
    for sentence in tqdm(sentences, position=0, leave=True):
      ids, masks, segments = create_single_input(sentence, tokenizer, MAX_SEQ_LEN-2) # why -2 ? ans: reserved for [CLS] & [SEP]
      # assert len(ids) == MAX_SEQ_LEN
      # assert len(masks) == MAX_SEQ_LEN
      # assert len(segments) == MAX_SEQ_LEN
      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)
    return [np.asarray(input_ids, dtype=np.int32), np.asarray(input_masks, dtype=np.int32), np.asarray(input_segments, dtype=np.int32)]

In [17]:
# two outputs: a pooled_output of shape [batch_size, 768] with representations for the entire input sequences 
# and a sequence_output of shape [batch_size, max_seq_length, 768] with representations for each input token (in context).

def nlp_model(bert_base):
    # Load the pre-trained BERT base model
    bert_layer = hub.KerasLayer(handle=bert_base, trainable=True)  
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")

    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    
    x = Dense(units=768, activation='relu')(pooled_output) # hidden layer 
    x = Dropout(0.15)(x) 
    outputs = Dense(2, activation="softmax")(x) # output layer

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [18]:
# hyper-parameters
BATCH_SIZE = 8
EPOCHS = 1
MAX_SEQ_LEN = 500

# model construction (we construct model first inorder to use bert_layer's tokenizer)
bert_base = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
model = nlp_model(bert_base) 
model.summary()

# Create examples for training and testing
df = df.sample(frac=1) # Shuffle the dataset
# we would like to use bert tokenizer; therefore, chech model.summary() and find the index of bert_layer
tokenizer = create_tonkenizer(model.layers[3])

# gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
# tf.io.gfile.listdir(gs_folder_bert)
# tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file = os.path.join(gs_folder_bert, "vocab.txt"), do_lower_case=True)

# create training data and testing data
x_train = convert_sentences_to_features(df['review'][:40000], tokenizer, MAX_SEQ_LEN)
x_valid = convert_sentences_to_features(df['review'][40000:45000], tokenizer, MAX_SEQ_LEN)
x_test = convert_sentences_to_features(df['review'][45000:], tokenizer, MAX_SEQ_LEN)
df['sentiment'].replace('positive', 1., inplace=True)
df['sentiment'].replace('negative', 0., inplace=True)
one_hot_encoded = to_categorical(df['sentiment'].values)
y_train = one_hot_encoded[:40000]
y_valid = one_hot_encoded[40000:45000]
y_test =  one_hot_encoded[45000:]

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 500)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 500)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 500)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_ids[0][0]',              
                                 (None, 500, 768)]                'input_masks[0][0]',        

100%|██████████| 5000/5000 [00:15<00:00, 314.47it/s]


In [19]:
# load the pretrained nlp_model
from tensorflow.keras.models import load_model
imdb_model = load_model('/content/drive/MyDrive/Presidential/imdb_bert.h5', custom_objects={'KerasLayer': hub.KerasLayer})

In [20]:
tinubu = pd.read_csv("/content/drive/MyDrive/Presidential/tinubuweets.csv")
obi = pd.read_csv("/content/drive/MyDrive/Presidential/mntweets.csv")

In [21]:
tinubu

,Unnamed: 0,Date,User,Tweet,Location,Hashtags
0,0,2023-02-23 23:59:43+00:00,daviesilva36,@doneghluis @ruffydfire Tinubu need to see this,NaN,NaN
1,1,2023-02-23 23:59:29+00:00,_ZainabGarba,@ZainabLadan5 Allah kadai yasan meh Tinubu ya baka Muhammad 😂😂,Nigeria,NaN
2,2,2023-02-23 23:58:56+00:00,Arinzecom1981,@fkeyamo Breakings News: \nTinubu rushed out of the Country for an undisclosed illness.,Nigeria,NaN
3,3,2023-02-23 23:58:51+00:00,ConsultingElvis,@fkeyamo Festus should knw that Eko Atlantic is privately owned and Tinubu can't take credit for that abandoned project..,NaN,NaN
4,4,2023-02-23 23:57:44+00:00,Hareef_TFL,The sacrosanct Mandate. A new Nigeria where possibilities are manifested by reinforcing the will that has in the past been actualized; now the magnitude is Federal. The progressive resolve to a renewed hope is backed by a Candidate of global repute; Asiwaju Bola Ahmed Tinubu. ♾ https://t.co/r7tfwATFZo,Trenches,NaN
...,...,...,...,...,...,...
99997,99995,2023-02-19 07:59:12+00:00,daddy_zert,What is the one thing that you want to do and you will always be proud of???\n\nOsinbajo ~ Rema ~ Obidients ~ Nigerians ~ Mc Oluomo ~ Bola Ahmed Tinubu,Nigeria,NaN
99998,99996,2023-02-19 07:59:09+00:00,telled404,February 25th Ote ma di ola fun Asiwaju Bola Ahmed Tinubu …. Gbo gbo ero wan ire loma jasi …. Asiwaju for President vote APC https://t.co/zmI94zCt9r,Abuja .Nigeria,NaN
99999,99997,2023-02-19 07:59:08+00:00,justin_obongha,@RebeccaAdeolu @officialABAT Tinubu has money already,"Abuja, Nigeria",NaN
100000,99998,2023-02-19 07:59:05+00:00,zainluxury_,"Newly restocked in sizes 38……….40 Price :27,000\n\nCaramel | Mandy | Akwuzu SARS | Asiwaju Bola Ahmed Tinubu | White Lion | Akwuzu SARS | 2face | Caramel | Atiku and Obi https://t.co/lh1uhz5pOn","Lagos, Nigeria",NaN


In [22]:
obi.head()

,Unnamed: 0,Date,User,Tweet,Location,Hashtags
0,0,2023-02-21 23:56:20+00:00,MRMELOD34742931,"Let's support the best candidates for Nigerians, and Nigeria as a whole,this is the time we have to work together to improve our situations in Nig,\nWe can make Nigeria a better place.Pls we urge all Nigeria's to vote wisely and set aside tribalism #PeterObi4President2023",United States,['PeterObi4President2023']
1,1,2023-02-21 23:50:40+00:00,Dollar1i,@TalentedFBG And we don’t care! You made a good choice!! #PeterObi4President2023 #ObiDatti2023 #OBIDIENT,NaN,"['PeterObi4President2023', 'ObiDatti2023', 'OBIDIENT']"
2,2,2023-02-21 23:48:18+00:00,_patrioticguy,Did Peter Obi actually went to the South East to campaign on a Monday?\n\nAbeg make una share receipts\n\n#PeterObi4President2023 https://t.co/OcgQawnxbH,Nigeria,['PeterObi4President2023']
3,3,2023-02-21 23:34:12+00:00,emmanuelkudayah,@Herdunney1 @NaijaWatch Glo Network #PeterObi4President2023,"Lagos, Nigeria",['PeterObi4President2023']
4,4,2023-02-21 23:25:44+00:00,dblacqboi,A vote for Peter obi is a vote for Good deeds if you are doing otherwise you are Evil. #daddyGO #PeterObi4President2023 #loveisland https://t.co/1dNbBEb1t4,"Lagos, Nigeria","['daddyGO', 'PeterObi4President2023', 'loveisland']"


In [23]:
import re
def preproc(tweets):
    processed_features = []

    for sentence in range(0, len(tweets)):
        # Remove all the special characters
        processed_feature = re.sub(r'\W', ' ', str(tweets[sentence]))

        # remove all single characters
        processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

        # Remove single characters from the start
        processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)

        # Substituting multiple spaces with single space
        processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

        # Removing prefixed 'b'
        processed_feature = re.sub(r'^b\s+', '', processed_feature)

        # Converting to Lowercase
        processed_feature = processed_feature.lower()

        processed_features.append(processed_feature)

    return processed_features

In [24]:
tinubu["Text"] = preproc(tinubu["Tweet"])
obi["Text"] = preproc(obi["Tweet"])

In [25]:
tinubu_textseq = convert_sentences_to_features(tinubu["Text"][:], tokenizer, MAX_SEQ_LEN)
tinubu_predicted_rating = np.argmax(imdb_model.predict(tinubu_textseq), axis=1)
tinubu_predicted_rating


 12%|█▏        | 11802/100002 [00:06<00:45, 1924.15it/s]


KeyboardInterrupt: ignored

In [ ]:
tinubu_pred = pd.DataFrame(tinubu_predicted_rating,columns = ["Sentiment"])

In [ ]:
tinubu["Sentiment"] = tinubu_pred

In [ ]:
tinubu.head(10)

,Unnamed: 0,Date,User,Tweet,Location,Hashtags,Text,Sentiment
0,0,2023-02-23 23:59:43+00:00,daviesilva36,@doneghluis @ruffydfire Tinubu need to see this,NaN,NaN,doneghluis ruffydfire tinubu need to see this,1
1,1,2023-02-23 23:59:29+00:00,_ZainabGarba,@ZainabLadan5 Allah kadai yasan meh Tinubu ya baka Muhammad 😂😂,Nigeria,NaN,zainabladan5 allah kadai yasan meh tinubu ya baka muhammad,1
2,2,2023-02-23 23:58:56+00:00,Arinzecom1981,@fkeyamo Breakings News: \nTinubu rushed out of the Country for an undisclosed illness.,Nigeria,NaN,fkeyamo breakings news tinubu rushed out of the country for an undisclosed illness,1
3,3,2023-02-23 23:58:51+00:00,ConsultingElvis,@fkeyamo Festus should knw that Eko Atlantic is privately owned and Tinubu can't take credit for that abandoned project..,NaN,NaN,fkeyamo festus should knw that eko atlantic is privately owned and tinubu can take credit for that abandoned project,1
4,4,2023-02-23 23:57:44+00:00,Hareef_TFL,The sacrosanct Mandate. A new Nigeria where possibilities are manifested by reinforcing the will that has in the past been actualized; now the magnitude is Federal. The progressive resolve to a renewed hope is backed by a Candidate of global repute; Asiwaju Bola Ahmed Tinubu. ♾ https://t.co/r7tfwATFZo,Trenches,NaN,the sacrosanct mandate new nigeria where possibilities are manifested by reinforcing the will that has in the past been actualized now the magnitude is federal the progressive resolve to renewed hope is backed by candidate of global repute asiwaju bola ahmed tinubu https co r7tfwatfzo,1


In [ ]:
tinubuu

,Unnamed: 0,Date,User,Tweet,Location,Hashtags,Text,Sentiment
0,0,2023-02-23 23:59:43+00:00,daviesilva36,@doneghluis @ruffydfire Tinubu need to see this,NaN,NaN,doneghluis ruffydfire tinubu need to see this,1.0
1,1,2023-02-23 23:59:29+00:00,_ZainabGarba,@ZainabLadan5 Allah kadai yasan meh Tinubu ya baka Muhammad 😂😂,Nigeria,NaN,zainabladan5 allah kadai yasan meh tinubu ya baka muhammad,1.0
2,2,2023-02-23 23:58:56+00:00,Arinzecom1981,@fkeyamo Breakings News: \nTinubu rushed out of the Country for an undisclosed illness.,Nigeria,NaN,fkeyamo breakings news tinubu rushed out of the country for an undisclosed illness,1.0
3,3,2023-02-23 23:58:51+00:00,ConsultingElvis,@fkeyamo Festus should knw that Eko Atlantic is privately owned and Tinubu can't take credit for that abandoned project..,NaN,NaN,fkeyamo festus should knw that eko atlantic is privately owned and tinubu can take credit for that abandoned project,1.0
4,4,2023-02-23 23:57:44+00:00,Hareef_TFL,The sacrosanct Mandate. A new Nigeria where possibilities are manifested by reinforcing the will that has in the past been actualized; now the magnitude is Federal. The progressive resolve to a renewed hope is backed by a Candidate of global repute; Asiwaju Bola Ahmed Tinubu. ♾ https://t.co/r7tfwATFZo,Trenches,NaN,the sacrosanct mandate new nigeria where possibilities are manifested by reinforcing the will that has in the past been actualized now the magnitude is federal the progressive resolve to renewed hope is backed by candidate of global repute asiwaju bola ahmed tinubu https co r7tfwatfzo,1.0


In [ ]:
obi_textseq = convert_sentences_to_features(tinubu["Text"][:], tokenizer, MAX_SEQ_LEN)
obi_predicted_rating = np.argmax(imdb_model.predict(obi_textseq), axis=1)
obi_predicted_rating

100%|██████████| 5/5 [00:00<00:00, 1489.03it/s]


1/1 [==============================] - 6s 6s/step


array([1, 1, 1, 1, 1])

In [ ]:
 obi_pred = pd.DataFrame(obi_predicted_rating,columns = ["Sentiment"])

In [ ]:
obi["Sentiment"] = obi_pred

In [ ]:
obi